In [ ]:
# Desarrollo de un modelo YOLO11

import os
import sys
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import VOCDetection
from torchvision.models import resnet50
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
  
    

In [ ]:
from ultralytics import YOLO

In [ ]:

from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

# Run inference with the YOLO11n model on the 'bus.jpg' image

#results = model("C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/YOLOV11/data/bus.jpg")
results = model("C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/YOLOV11/data/bus2.jpg")
#results = model("C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/YOLOV11/data/bus.jpg")
#results = model("C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/YOLOV11/data/bus.jpg")



In [ ]:
path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/datasets/'
path_coco8_data = path + 'coco8/'


In [ ]:
# Entrenamiento de un modelo YOLO11 con un dataset personalizado

from ultralytics import YOLO

# Cargar el modelo YOLO11 preentrenado
model = YOLO("yolov11.pt")

# Entrenar con tu dataset
model.train(data="ruta/a/data.yaml", epochs=100, imgsz=640)

In [ ]:
# Realizar inferencia con el modelo YOLO11 entrenado sobre imagen

from ultralytics import YOLO

model = YOLO("yolov11.pt")
results = model("ruta/a/imagen.jpg")
results.show()
results = model('ruta/a/mi_imagen.jpg', save=True)

In [ ]:
# Realizar inferencia con el modelo YOLO11 entrenado sobre video

from ultralytics import YOLO

model = YOLO("yolov11.pt")
results = model('ruta/a/mi_video.mp4')
results.show()  # Muestra el video con detecciones en tiempo real
results = model('ruta/a/mi_video.mp4', save=True)

In [ ]:
! pip install ultralytics opencv-python

In [ ]:
# Version Genie de Cosine del dia 09/06/2025

import cv2
from ultralytics import YOLO

# 1. Cargar el modelo YOLOv11 preentrenado
model = YOLO('yolo11n.pt')

# 2. Leer el video de entrada
video_path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/comma_small.mp4'
cap = cv2.VideoCapture(video_path)

# 3. Preparar el writer para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('video_con_detecciones.mp4', fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 4. Diccionario para guardar trayectorias por ID
trajectories = dict()

# 5. Procesar frame a frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv11 inferencia con tracking activado
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")  # Usa tracking basado en ByteTrack

    # Filtrar solo los vehículos (revisa clases: car, bus, truck, etc.)
    for det in results:
        if det.boxes is not None:
            for box in det.boxes:
                cls_id = int(box.cls[0])
                track_id = int(box.id[0]) if box.id is not None else None
                # Ajusta según las clases de vehículos en YOLOv11
                # 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
                # 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                # 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                # 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
                # 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
                # 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
                if model.names[cls_id] in ['person', 'bycicle', 'car', 'motorcycle', 'bus', 'truck', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench']:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    # Dibujar bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(frame, f"{model.names[cls_id]} ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

                    # Guardar centro del bounding box para la trayectoria
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    if track_id not in trajectories:
                        trajectories[track_id] = []
                    trajectories[track_id].append((cx, cy))

    # Dibujar trazas de cada objeto
    for track_id, points in trajectories.items():
        for i in range(1, len(points)):
            cv2.line(frame, points[i-1], points[i], (255,0,0), 2)

    # Escribir frame al video de salida
    out.write(frame)

# Liberar recursos
cap.release()
out.release()
print("¡Video generado con bounding boxes y trayectorias guardado como 'video_con_detecciones.mp4'!")


0: 384x640 1 traffic light, 74.2ms
Speed: 2.4ms preprocess, 74.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.4ms
Speed: 1.6ms preprocess, 48.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 42.7ms
Speed: 0.9ms preprocess, 42.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.2ms
Speed: 1.6ms preprocess, 46.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 44.7ms
Speed: 1.1ms preprocess, 44.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.9ms
Speed: 0.9ms preprocess, 44.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.7ms
Speed: 1.0ms preprocess, 46.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.6ms
Speed: 1.5ms preprocess, 47.6ms i

Notas importantes:
Cambia ['car', 'truck', 'bus', 'motorcycle'] según las clases que YOLOv11 tenga para vehículos (puedes imprimir model.names para verlas).
El tracking usa ByteTrack (incluido en Ultralytics). Puedes cambiar el tracker si lo deseas (ver docs Ultralytics).
Este código guarda el video con bbboxes y trayectorias en el mismo tamaño y fps que el original.

In [2]:
print(model.names)  # Imprime las clases del modelo YOLOv11

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [ ]:
# Version Pablo Izquierdo del dia 09/06/2025

from ultralytics import YOLO
import cv2

# Cargar modelo YOLOv11
model = YOLO("yolo11n.pt") # Asegúrate de tener este archivo en el directorio

# Ruta del video de entrada
video_path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/comma_small.mp4' # Reemplaza con la ruta a tu video

# Inicializar captura de video
cap = cv2.VideoCapture(video_path)
merged_tracks = {} # Diccionario para almacenar trayectorias

# Obtener propiedades del video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30 # FPS estimado si es 0

# Configurar escritor de video
out = cv2.VideoWriter('C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/tracking_coches_test.avi',
cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
print("Procesando video...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Ejecutar detección y seguimiento en el fotograma actual
    results = model.track(frame, persist=True, conf=0.5, verbose=False)[0]
    boxes = results.boxes
    unique_ids = set()
    for box in boxes:
        if int(box.cls[0]) not in [2, 5, 7] or box.id is None:
            continue
        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        unique_ids.add(track_id)

        # Almacenar trayectoria
        if track_id not in merged_tracks:
            merged_tracks[track_id] = []
        merged_tracks[track_id].append(center)

        # Dibujar caja delimitadora en rojo
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Dibujar trayectoria en blanco
        for pt in merged_tracks[track_id]:
            cv2.circle(frame, (int(pt[0]), int(pt[1])), 2, (255, 255, 255), -1)
    
    # Mostrar el fotograma procesado (opcional)
    cv2.imshow("Detección en video", frame)

    # Guardar el fotograma en el video de salida
    out.write(frame)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Procesamiento completado. Video guardado como tracking_coches.avi")



Procesando video...
WARNING 'source' is missing. Using 'source=C:\Users\gtoma\anaconda3\envs\pytorch_13042025\Lib\site-packages\ultralytics\assets'.


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
